In [ ]:
import sys

sys.path.append("../..")
import torch
import numpy as np

In [ ]:
from xlstm.xlstm_large.model import xLSTMLargeConfig, xLSTMLarge
from mlstm_kernels.torch import get_available_mlstm_step_kernels, get_available_mlstm_kernels, get_available_mlstm_sequence_kernels

In [ ]:
get_available_mlstm_kernels(), get_available_mlstm_step_kernels(), get_available_mlstm_sequence_kernels()

In [ ]:
xlstm_config = xLSTMLargeConfig(
    embedding_dim=512,
    num_heads=4,
    num_blocks=6,
    vocab_size=2048,
    return_last_states=True,
    mode="inference",
    chunkwise_kernel="chunkwise--triton_limit_chunk",
    sequence_kernel="native_sequence__triton",
    step_kernel="triton",
)

In [ ]:
xlstm = xLSTMLarge(xlstm_config)

In [ ]:
xlstm

In [ ]:
xlstm = xlstm.to("cuda")

In [ ]:
input = torch.randint(0, 2048, (3, 256)).to("cuda")
input.shape

In [ ]:
out = xlstm(input)

In [ ]:
if len(out) == 2:
    out, state = out

In [ ]:
out.shape

In [ ]:
state.keys()

In [ ]:
len(state), len(state[0])

In [ ]:
input[:, 0:1].shape, input.shape

In [ ]:
step_out, step_state = xlstm(input[:, 0:1], state)

In [ ]:
step_out.shape

In [ ]:
out_chunkwise, last_state_chunkwise = xlstm(input)

In [ ]:
out_steps = []
state = None
for i in range(input.shape[1]):
    out_step, state = xlstm(input[:, i:i + 1], state)
    out_steps.append(out_step)

In [ ]:
out_steps = torch.cat(out_steps, dim=1)

In [ ]:
out_steps.shape, out_chunkwise.shape

In [ ]:
(out_chunkwise - out_steps).abs().max()

In [ ]:
torch.allclose(out_chunkwise, out_steps, atol=7e-2, rtol=1e-3)

In [ ]:
list(xlstm.state_dict().keys())